# Implementing RNN to predict the Cryptocurrency

Part - I

In [26]:
import pandas as pd
import os
from collections import deque
import random
from sklearn import preprocessing
import numpy as np
import time

In [4]:
df = pd.read_csv("crypto_data/BTC-USD.csv", names = ["time", "low", "high", "open", "close", "volume"])

In [5]:
df.head(10)

,time,low,high,open,close,volume
0,1528968660,6489.549805,6489.560059,6489.560059,6489.549805,0.587100
1,1528968720,6487.370117,6489.560059,6489.549805,6487.379883,7.706374
2,1528968780,6479.410156,6487.370117,6487.370117,6479.410156,3.088252
3,1528968840,6479.410156,6479.419922,6479.419922,6479.410156,1.404100
4,1528968900,6475.930176,6479.979980,6479.410156,6479.979980,0.753000
5,1528968960,6477.959961,6480.000000,6477.959961,6480.000000,1.490900
6,1528969020,6477.220215,6480.000000,6479.990234,6477.220215,2.731950
7,1528969080,6477.220215,6480.000000,6477.220215,6480.000000,2.174240
8,1528969140,6479.990234,6479.990234,6479.990234,6479.990234,0.903100
9,1528969200,6477.259766,6479.990234,6479.990234,6478.660156,3.258786


### Creating a DataFame containing all desired values

In [6]:
main_df = pd.DataFrame()

In [7]:
ratios = ["BTC-USD", "LTC-USD", "BCH-USD", "ETH-USD"] 
for ratio in ratios:  # begin iteration
    print(ratio)
    dataset = f'crypto_data/{ratio}.csv'  # path to the file.
    df = pd.read_csv(dataset, names=['time', 'low', 'high', 'open', 'close', 'volume'])  # read in specific file

    # rename volume and close:
    df.rename(columns={"close": f"{ratio}_close", "volume": f"{ratio}_volume"}, inplace=True)

    df.set_index("time", inplace=True)  # set time as index 
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]  # ignore the other columns besides price and volume

    if len(main_df)==0:  # if the dataframe is empty
        main_df = df  # then it's just the current df
    else:  # otherwise, join this data to the main one
        main_df = main_df.join(df)

BTC-USD
LTC-USD
BCH-USD
ETH-USD


In [ ]:
main_df.fillna(method="ffill", inplace=True)  # if there are gaps in data, use previously known values
main_df.dropna(inplace=True)

In [8]:
print(main_df.head())

            BTC-USD_close  BTC-USD_volume  LTC-USD_close  LTC-USD_volume  \
time                                                                       
1528968660    6489.549805        0.587100      96.580002        9.647200   
1528968720    6487.379883        7.706374      96.660004      314.387024   
1528968780    6479.410156        3.088252      96.570000       77.129799   
1528968840    6479.410156        1.404100      96.500000        7.216067   
1528968900    6479.979980        0.753000      96.389999      524.539978   

            BCH-USD_close  BCH-USD_volume  ETH-USD_close  ETH-USD_volume  
time                                                                      
1528968660     871.719971        5.675361            NaN             NaN  
1528968720     870.859985       26.856577      486.01001       26.019083  
1528968780     870.099976        1.124300      486.00000        8.449400  
1528968840     870.789978        1.749862      485.75000       26.994646  
1528968900     87

Part - II

In [9]:
SEQ_LEN = 60  # how long of a preceeding sequence to collect for RNN
FUTURE_PERIOD_PREDICT = 3  # how far into the future are we trying to predict?
RATIO_TO_PREDICT = "LTC-USD"

In [10]:
def classify(current, future):
    if float(future) > float(current):
        return 1 #  Buy
    else:
        return 0 # Sell

#### Adding Future and target columns in the dataframe

In [11]:
main_df['future'] = main_df[f'{RATIO_TO_PREDICT}_close'].shift(-FUTURE_PERIOD_PREDICT)

In [12]:
main_df['target'] = list(map(classify, main_df[f'{RATIO_TO_PREDICT}_close'], main_df['future']))

In [13]:
print(main_df.head())

            BTC-USD_close  BTC-USD_volume  LTC-USD_close  LTC-USD_volume  \
time                                                                       
1528968660    6489.549805        0.587100      96.580002        9.647200   
1528968720    6487.379883        7.706374      96.660004      314.387024   
1528968780    6479.410156        3.088252      96.570000       77.129799   
1528968840    6479.410156        1.404100      96.500000        7.216067   
1528968900    6479.979980        0.753000      96.389999      524.539978   

            BCH-USD_close  BCH-USD_volume  ETH-USD_close  ETH-USD_volume  \
time                                                                       
1528968660     871.719971        5.675361            NaN             NaN   
1528968720     870.859985       26.856577      486.01001       26.019083   
1528968780     870.099976        1.124300      486.00000        8.449400   
1528968840     870.789978        1.749862      485.75000       26.994646   
1528968900 

In [14]:
times = sorted(main_df.index.values)  # get the times
last_5pct = sorted(main_df.index.values)[-int(0.05*len(times))]  # get the last 5% of the times

In [15]:
validation_main_df = main_df[(main_df.index >= last_5pct)]  # make the validation data where the index is in the last 5%
main_df = main_df[(main_df.index < last_5pct)]

Balancing and Normalizing Data

Part - III

In [23]:
def preprocess_df(df):
    df = df.drop("future", 1)  # don't need this anymore.

    for col in df.columns:  # go through all of the columns
        if col != "target":  # normalize all ... except for the target itself!
            df[col] = df[col].pct_change()  # pct change "normalizes" the different currencies (each crypto coin has vastly diff values, we're really more interested in the other coin's movements)
            df.dropna(inplace=True)  # remove the nas created by pct_change
            df[col] = preprocessing.scale(df[col].values)  # scale between 0 and 1.

    df.dropna(inplace=True)  # cleanup again... jic.


    sequential_data = []  # this is a list that will CONTAIN the sequences
    prev_days = deque(maxlen=SEQ_LEN)  # These will be our actual sequences. They are made with deque, which keeps the maximum length by popping out older values as new ones come in

    for i in df.values:  # iterate over the values
        prev_days.append([n for n in i[:-1]])  # store all but the target
        if len(prev_days) == SEQ_LEN:  # make sure we have 60 sequences!
            sequential_data.append([np.array(prev_days), i[-1]])  # append those bad boys!

    random.shuffle(sequential_data)  # shuffle for good measure.

    buys = []  # list that will store our buy sequences and targets
    sells = []  # list that will store our sell sequences and targets

    for seq, target in sequential_data:  # iterate over the sequential data
        if target == 0:  # if it's a "not buy"
            sells.append([seq, target])  # append to sells list
        elif target == 1:  # otherwise if the target is a 1...
            buys.append([seq, target])  # it's a buy!

    random.shuffle(buys)  # shuffle the buys
    random.shuffle(sells)  # shuffle the sells!

    lower = min(len(buys), len(sells))  # what's the shorter length?

    buys = buys[:lower]  # make sure both lists are only up to the shortest length.
    sells = sells[:lower]  # make sure both lists are only up to the shortest length.

    sequential_data = buys + sells  # add them together
    random.shuffle(sequential_data)  # another shuffle, so the model doesn't get confused with all 1 class then the other.

    X = []
    y = []

    for seq, target in sequential_data:  # going over our new sequential data
        X.append(seq)  # X is the sequences
        y.append(target)  # y is the targets/labels (buys vs sell/notbuy)

    return np.array(X), y  # return X and y...and make X a numpy array!

In [24]:
train_x, train_y = preprocess_df(main_df)
validation_x, validation_y = preprocess_df(validation_main_df)

<ipython-input-23-92c4de6df35c>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop("future", 1)  # don't need this anymore.
<ipython-input-23-92c4de6df35c>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop("future", 1)  # don't need this anymore.


In [25]:
print(f"train data: {len(train_x)} validation: {len(validation_x)}")
print(f"Dont buys: {train_y.count(0)}, buys: {train_y.count(1)}")
print(f"VALIDATION Dont buys: {validation_y.count(0)}, buys: {validation_y.count(1)}")

train data: 69188 validation: 3062
Dont buys: 34594, buys: 34594
VALIDATION Dont buys: 1531, buys: 1531


Part - IV

New Constants

In [42]:
EPOCHS = 10  # how many passes through our data
BATCH_SIZE = 64  # how many batches? Try smaller batch if you're getting OOM (out of memory) errors.
NAME = f"{RATIO_TO_PREDICT}-{SEQ_LEN}-SEQ-{FUTURE_PERIOD_PREDICT}-PRED-{int(time.time())}"

Model

In [43]:
train_x = np.asarray(train_x)
train_y = np.asarray(train_y)
validation_x = np.asarray(validation_x)
validation_y = np.asarray(validation_y)

In [35]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint

In [45]:
model = Sequential()
model.add(LSTM(128, input_shape=(train_x.shape[1:]), activation='relu', return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())  #normalizes activation outputs, same reason you want to normalize your input data.

model.add(LSTM(128, activation='relu', return_sequences=True))
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(LSTM(128, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2, activation='softmax'))

opt = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)

# Compile model
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy']
)

tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))
filepath = "RNN_Final-{epoch:02d}-{val_accuracy:.3f}"  # unique file name that will include the epoch and the validation acc for that epoch
checkpoint = ModelCheckpoint("models/{}.model".format(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')) # saves only the best ones

# Train model
history = model.fit(
    train_x, train_y,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(validation_x, validation_y),
    callbacks=[tensorboard, checkpoint],
)

Epoch 1/10
1082/1082 [==============================] - ETA: 0s - loss: 0.7170 - accuracy: 0.5022INFO:tensorflow:Assets written to: models\RNN_Final-01-0.501.model\assets


1082/1082 [==============================] - 237s 214ms/step - loss: 0.7170 - accuracy: 0.5022 - val_loss: 0.6967 - val_accuracy: 0.5010
Epoch 2/10
1082/1082 [==============================] - ETA: 0s - loss: 0.6931 - accuracy: 0.5095INFO:tensorflow:Assets written to: models\RNN_Final-02-0.501.model\assets


INFO:tensorflow:Assets written to: models\RNN_Final-02-0.501.model\assets


1082/1082 [==============================] - 268s 247ms/step - loss: 0.6931 - accuracy: 0.5095 - val_loss: 0.7088 - val_accuracy: 0.5007
Epoch 3/10
1082/1082 [==============================] - ETA: 0s - loss: 0.6931 - accuracy: 0.5116INFO:tensorflow:Assets written to: models\RNN_Final-03-0.503.model\assets


INFO:tensorflow:Assets written to: models\RNN_Final-03-0.503.model\assets


...

1082/1082 [==============================] - 288s 267ms/step - loss: 0.6928 - accuracy: 0.5121 - val_loss: 0.6943 - val_accuracy: 0.5059
Epoch 9/10
1082/1082 [==============================] - ETA: 0s - loss: 0.6927 - accuracy: 0.5148INFO:tensorflow:Assets written to: models\RNN_Final-09-0.494.model\assets


INFO:tensorflow:Assets written to: models\RNN_Final-09-0.494.model\assets


1082/1082 [==============================] - 341s 315ms/step - loss: 0.6927 - accuracy: 0.5148 - val_loss: 0.6971 - val_accuracy: 0.4944
Epoch 10/10
1082/1082 [==============================] - ETA: 0s - loss: 0.6928 - accuracy: 0.5117INFO:tensorflow:Assets written to: models\RNN_Final-10-0.500.model\assets


INFO:tensorflow:Assets written to: models\RNN_Final-10-0.500.model\assets


1082/1082 [==============================] - 271s 250ms/step - loss: 0.6928 - accuracy: 0.5117 - val_loss: 0.6959 - val_accuracy: 0.5003


In [46]:
# Score model
score = model.evaluate(validation_x, validation_y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
# Save model
model.save("models/{}".format(NAME))

Test loss: 0.6958659887313843
Test accuracy: 0.5003265738487244
INFO:tensorflow:Assets written to: models/LTC-USD-60-SEQ-3-PRED-1646247353\assets


INFO:tensorflow:Assets written to: models/LTC-USD-60-SEQ-3-PRED-1646247353\assets
